In [1]:
from keras.optimizers import Adam,Nadam
import numpy as np

In [2]:
# Setting Up Hyperparameters
np.random.seed(10)
noise_dim = 100

batch_size = 16
steps_per_epoch = 3750
epochs = 10

save_path = "fcgan_images"

img_rows, img_cols, channels = 28,28,1

optimizer = Adam(0.0002, 0.5)

In [3]:
# Loading dataset
from keras.datasets import mnist
import os

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5)/127.5
x_train = x_train.reshape(-1,img_rows*img_cols*channels)

if not os.path.isdir(save_path):
    os.mkdir(save_path)

In [4]:
# implementing generator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU

def create_generator():
    generator = Sequential()
    
    generator.add(Dense(256, input_dim=noise_dim))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(img_rows*img_cols*channels, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return generator

In [5]:
# implementing discriminator

def create_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(1024, input_dim=img_rows*img_cols*channels))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return discriminator

In [6]:
# instanciating G&D
discriminator = create_discriminator()
generator = create_generator()

discriminator.trainable = False

In [7]:
# setting up the combined model
from keras.layers import Input
from keras.models import Model

gan_input = Input(shape=(noise_dim,))
fake_image = generator(gan_input)

gan_output = discriminator(fake_image)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [8]:
# implementing image viewer
import matplotlib.pyplot as plt
%matplotlib inline
def show_images(noise, epoch=None):
    generated_images = generator.predict(noise)
    plt.figure(figsize=(10, 10))
    
    for i, image in enumerate(generated_images):
        plt.subplot(10, 10, i+1)
        if channels == 1:
            plt.imshow(image.reshape((img_rows, img_cols)), cmap='gray')
        else:
            plt.imshow(image.reshape((img_rows, img_cols, channels)))
        plt.axis('off')
    
    plt.tight_layout()
    
    if epoch != None and save_path != None:
        plt.savefig(f'{save_path}/gan-images_epoch-{epoch}.png')


In [ ]:
# trainging the model
static_noise = np.random.normal(0, 1, size=(100, noise_dim))

for epoch in range(epochs):
    for batch in range(steps_per_epoch):
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        fake_x = generator.predict(noise)
        
        real_x = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
        
        x = np.concatenate((real_x, fake_x))
        
        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9
        
        d_loss = discriminator.train_on_batch(x, disc_y)
        
        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)
        
    print(f'Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}')
#     if epoch%10 == 0:
    show_images(static_noise, epoch)

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step


...

1/1 [==============================] - 0s 237ms/step


1/1 [==============================] - 0s 140ms/step


1/1 [==============================] - 0s 155ms/step


1/1 [==============================] - 0s 106ms/step


1/1 [==============================] - 0s 147ms/step
